### PPL 계산하기

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.models import vgg16
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import random


# 이미지 로드 및 전처리
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # VGG16 입력 크기
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image).unsqueeze(0)

# 특징 벡터 추출
def extract_features(model, image_tensor):
    with torch.no_grad():
        features = model(image_tensor)
    return features

# Euclidean 거리 계산
def euclidean_distance(a, b):
    return torch.norm(a - b).item()

# Perceptual Path Length 계산
def perceptual_path_length(images, model):
    features = [extract_features(model, load_image(img_path)).flatten() for img_path in images]
    distances = [euclidean_distance(features[i], features[i + 1]) for i in range(len(features) - 1)]
    return sum(distances), distances

# 이미지 경로 설정
image_paths = ['dog_t_ep1.jpg','dog_t.jpg','dog_t_ep2.jpg']  # 이미지 경로를 적절히 수정하세요.

# VGG16 모델 로드
model = vgg16(pretrained=True).eval()
model = torch.nn.Sequential(*list(model.children())[:-1])  # 마지막 분류 층 제거

# PPL 계산
total_ppl, distances = perceptual_path_length(image_paths, model)
print(f"Total Perceptual Path Length: {total_ppl}")

# 각 막대의 색상 지정
colors = ["orange", "green"]

# 결과 시각화
plt.figure(figsize=(4, 8))
plt.bar(range(len(distances)), distances, color=colors, tick_label=[f'Path {i+1}' for i in range(len(distances))])
plt.xlabel('Image Pairs')
plt.ylabel('Perceptual Distance')
plt.title('Perceptual Path Length Between Images')
plt.show()
